In [1]:
import glob, sys
from pathlib import Path
from omegaconf import OmegaConf
from tqdm.auto import tqdm
from sm.prelude import M, O
from grams.inputs import LinkedTable
from grams.config import ROOT_DIR

In [2]:
from sm_widgets.widgets.annotator import Annotator, BatchAnnotator
from sm_widgets_integration.wikidata import get_qnode_db, get_ontclass_db, get_ontprop_db
from sm_widgets_integration.with_grams import convert_table

qnode_db = get_qnode_db(ROOT_DIR / "data/qnodes.db", proxy=True)
wdprop_db = get_ontclass_db(ROOT_DIR / "data/wdprops.db", proxy=True)
wdclass_db = get_ontprop_db(ROOT_DIR / "data/wdclasses.db", proxy=True)

In [3]:
cwd = "./v210514/"
table_ids = {Path(x).stem for x in glob.glob(cwd + 'tables/*.csv')}
inputs = []
for indir in glob.glob(cwd + 'wikidata/descriptions/*'):
    infile = M.get_latest_path(indir + '/version.json')
    table_id = Path(indir).name
    sms = [O.SemanticModel.from_json(x) for x in M.deserialize_json(infile)['sms']]
    table = LinkedTable.from_csv_file(cwd + f'tables/{table_id}.csv', cwd + f'wikidata/entities/{table_id}.tsv')
    inputs.append([table.id, "", convert_table(table), sms])

In [4]:
annotator = Annotator(qnode_db, wdclass_db, wdprop_db, 'http://www.wikidata.org/prop/P31',
                      dev=False,
                      savedir='', eshost='', embedded_error_log=True)
# annotator.render(same_tab=False)
bann = BatchAnnotator(annotator, dev=False)
bann.render(same_tab=False)

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='3261af24-03ea-4309-a087-44f5568bdae6')

Output()

<IPython.core.display.Javascript object>

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='26e29e80-96b4-4a3c-b73f-5a712496aa49')

<IPython.core.display.Javascript object>

In [5]:
import threading
threading.Thread(
    target=lambda: bann.set_data(inputs)
).start()